# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 2:13PM,247083,10,0086002909,ISDIN Corporation,Released
1,Sep 22 2022 2:13PM,247083,10,0086002925,ISDIN Corporation,Released
2,Sep 22 2022 2:13PM,247083,10,0086002940,ISDIN Corporation,Released
3,Sep 22 2022 2:13PM,247083,10,0086002948,ISDIN Corporation,Released
4,Sep 22 2022 2:13PM,247083,10,0086002960,ISDIN Corporation,Released
5,Sep 22 2022 2:13PM,247083,10,0086002961,ISDIN Corporation,Released
6,Sep 22 2022 2:13PM,247083,10,0086002975,ISDIN Corporation,Released
7,Sep 22 2022 2:13PM,247083,10,0086002983,ISDIN Corporation,Released
8,Sep 22 2022 2:13PM,247083,10,0086002991,ISDIN Corporation,Released
9,Sep 22 2022 2:13PM,247083,10,0086003133,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,247082,Released,1
25,247083,Released,10
26,247084,Released,1
27,247085,Released,1
28,247086,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247082,NaN,1.0
247083,NaN,10.0
247084,NaN,1.0
247085,NaN,1.0
247086,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246977,0.0,13.0
247005,9.0,2.0
247015,0.0,1.0
247026,40.0,1.0
247031,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246977,0,13
247005,9,2
247015,0,1
247026,40,1
247031,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246977,0,13
1,247005,9,2
2,247015,0,1
3,247026,40,1
4,247031,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246977,,13
1,247005,9,2
2,247015,,1
3,247026,40,1
4,247031,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation
10,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc."
11,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc."
12,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc."
13,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix
14,Sep 22 2022 2:07PM,247081,22,"NBTY Global, Inc."
15,Sep 22 2022 2:02PM,247080,19,"Methapharm, Inc."
16,Sep 22 2022 1:56PM,247079,19,"GUSA Granules USA, Inc."
17,Sep 22 2022 1:50PM,247078,19,"GUSA Granules USA, Inc."
19,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation,,10
1,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc.",,1
2,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc.",,1
3,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc.",,1
4,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix,,1
5,Sep 22 2022 2:07PM,247081,22,"NBTY Global, Inc.",,1
6,Sep 22 2022 2:02PM,247080,19,"Methapharm, Inc.",,1
7,Sep 22 2022 1:56PM,247079,19,"GUSA Granules USA, Inc.",,1
8,Sep 22 2022 1:50PM,247078,19,"GUSA Granules USA, Inc.",,2
9,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation,10,
1,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc.",1,
2,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc.",1,
3,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc.",1,
4,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix,1,
5,Sep 22 2022 2:07PM,247081,22,"NBTY Global, Inc.",1,
6,Sep 22 2022 2:02PM,247080,19,"Methapharm, Inc.",1,
7,Sep 22 2022 1:56PM,247079,19,"GUSA Granules USA, Inc.",1,
8,Sep 22 2022 1:50PM,247078,19,"GUSA Granules USA, Inc.",2,
9,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation,10,
1,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc.",1,
2,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc.",1,
3,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc.",1,
4,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation,10.0,NaN
1,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc.",1.0,NaN
2,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc.",1.0,NaN
3,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc.",1.0,NaN
4,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 2:13PM,247083,10,ISDIN Corporation,10.0,0.0
1,Sep 22 2022 2:11PM,247086,22,"NBTY Global, Inc.",1.0,0.0
2,Sep 22 2022 2:11PM,247085,22,"NBTY Global, Inc.",1.0,0.0
3,Sep 22 2022 2:11PM,247084,22,"NBTY Global, Inc.",1.0,0.0
4,Sep 22 2022 2:08PM,247082,10,Yusen – 3D Matrix,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2223392,37.0,10.0
12,247040,1.0,0.0
15,494060,2.0,40.0
16,494114,5.0,0.0
19,988276,5.0,0.0
20,741114,35.0,3.0
22,988336,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2223392,37.0,10.0
1,12,247040,1.0,0.0
2,15,494060,2.0,40.0
3,16,494114,5.0,0.0
4,19,988276,5.0,0.0
5,20,741114,35.0,3.0
6,22,988336,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,37.0,10.0
1,12,1.0,0.0
2,15,2.0,40.0
3,16,5.0,0.0
4,19,5.0,0.0
5,20,35.0,3.0
6,22,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,12,Released,1.0
2,15,Released,2.0
3,16,Released,5.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Executing,10.0,0.0,40.0,0.0,0.0,3.0,0.0
Released,37.0,1.0,2.0,5.0,5.0,35.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Executing,10.0,0.0,40.0,0.0,0.0,3.0,0.0
1,Released,37.0,1.0,2.0,5.0,5.0,35.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Executing,10.0,0.0,40.0,0.0,0.0,3.0,0.0
1,Released,37.0,1.0,2.0,5.0,5.0,35.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()